# HCMUS-UNIV-Stenography-MP3

Thực hiện: 2042051 - Nguyễn Thành Long

## I.Mục tiêu:

- Triển khai phương pháp Nhúng thông tin mật vào file MP3 thông qua phương pháp: **POST ENCONDING**
- Phương pháp nhúng tin mật vào các bit ít/không quan trọng trong vùng header.

## II.Tổng quát:

- MP3 là file tiêu chuẩn mở
- Phổ biến
- Phù hợp cho người dùng cuối
- File nén lossy giữ chất lượng âm thanh nhưng vẫn tối ưu không gian lưu trữ

## III.Phương pháp thực hiện

- Với phương pháp nhúng tin mật vào các bit trong header ta cần tìm hiểu về header của file mp3.

- Như đã biết thì mp3 là file âm thanh chia thành các phần nhỏ, mỗi phần nhỏ đó gọi là **FRAME**.

- Mỗi **FRAME** sẽ gồm 2 phần là Header và Audio Data.

### 1.Cấu trúc file MP3

- Nội dung file MP3 sẽ phụ thuộc vào lại encoding sử dụng, nhưng tổng quát có dạng sau:

<img src="img/mp3_1.jpg" alt="drawing" width="300"/>

- Nhưng sau này người ta muốn nhúng thêm các thông tin khác vào cho file mp3 để thuận tiện quảng bá tới người dùng hơn nên mới thiết kế ra TAG (ID3)

- ID3v1 (1996) : Được gán vào cuối file MP3 lưu trữ các meta data tối đa 128 bytes. 

<img src="img/id3v1_blocks.gif" alt="drawing" width="100"/>

- ID3v2 (1998) : Cảm thấy lưu trữ 128 bytes là chưa đủ nên với sự ra đời của ID3v2 thì metadata đã phong phú hơn rất nhiều, có thể lưu trữ tơí 256 MB. Phần này sẽ nằm ở đầu tệp tin MP3

<img src="img/id3v2_blocks.gif" alt="drawing" width="120"/>



- Nhưng đôi khi thì việc thêm TAG vào không "Hay" lắm nên TAG sẽ là 1 tùy chọn trong FILE MP3

- Tổng kết lại ta sẽ có cấu trúc file MP3 hoàn chỉnh sau:

<img src="img/mp3_2.jpg" alt="drawing" width="500"/>


### 2.Cấu trúc **HEADER** của 1 FRAME MP3

- HEADER sẽ gồm 4 Bytes (32 bit) thông tin được phân bổ như sau:
( mỗi 1 ô vuông là 1 bit)

<img src="img/mp3frame_blocks.gif" alt="drawing" width="100"/>

- Trong 32 bit này thì sẽ có những BIT ít quan trọng/ không quan trọng (quá trình decode không cần thiết) như là : private bit, copyright bit, home bit và emphasis bit chúng ta có thể lợi dụng để nhúng thông tin mật vào đây.

- Như vậy là có 5 bits mỗi frame chúng ta có thể nhúng dữ liệu

### 3.Thuật toán nhúng và trích xuất thông tin

1. Nhúng:

> Bước 1: Tìm offset Frame đầu tiên của file MP3

> Bước 2: Nhúng thông tin vào các bit đã xác định

> Bước 3: Tìm offset Frame tiếp theo và tiếp tục nhúng cho đến khi hết chuỗi bit msg

2. Trích xuất:

> Bước 1: Tìm offset Frame đầu tiên của file MP3

> Bước 2: Trích xuất thông tin từ các bit đã xác định

> Bước 3: Tìm offset Frame tiếp theo và tiếp tục trích xuất cho đến khi hết Frame


=> Sử dụng phương pháp chèn 100... vào cuối msg để dể dàng trích xuất

### 4.Kỹ thuật tìm offset của Frame đầu tiên và Loại bỏ TAG

- Việc loại bỏ các TAG ra khỏi quá trình nhúng là hết sức cần thiết, nó cần cho việc tìm offset của Frame đầu tiên.
<br>
<br>
- Chúng ta đã biết Tag ID3v1 nằm ở cuối file MP3, nhưng nó là optional nên chúng ta cần detect nó trước khi xóa.
- ID3v1 Tag có kích thước 128bytes và có 3 bytes đầu tiên là "TAG" và nên ta chỉ cần xem xét bytes array [-128:125] xem có phải "TAG" hay không và xóa đi nếu thỏa.

- Với Tag ID3v2 thì phức tạp hơn: theo như mô tả từ [https://id3.org/id3v2.4.0-structure](https://id3.org/id3v2.4.0-structure) thì như sau:

```
     +-----------------------------+
     |      Header (10 bytes)      |
     +-----------------------------+
     |       Extended Header       |
     | (variable length, OPTIONAL) |
     +-----------------------------+
     |   Frames (variable length)  |
     +-----------------------------+
     |           Padding           |
     | (variable length, OPTIONAL) |
     +-----------------------------+
     | Footer (10 bytes, OPTIONAL) |
     +-----------------------------+
```
1. ID3v2 header : Phần đầu tiên của ID3v2 là header chứa 10 bytes
```
     ID3v2/file identifier      "ID3"
     ID3v2 version              $04 00
     ID3v2 flags                %abcd0000
     ID3v2 size             4 * %0xxxxxxx
```
> 3 bytes đầu tiên luôn là "ID3"
>
> byte thứ 7->10 chứa **size** đây là kích thước của TAG (không tính header)<br>
> size được lưu trữ ở dạng 32 bit synchsafe integer <br>
> Chúng ta sẽ sử dụng hàm dưới để chuyển synchsafe sang int

In [84]:
def synchsafe2int(numb):
    return numb[0] << 21 | numb[1] << 14 | numb[2] << 7 | numb[3]

> Vậy nếu như tồn tại Tag ID3v2 để đến được offset Frame đầu tiên ta cần nhảy 1 đoạn là :

```synchsafe2int(sizeTag) + 10 bytes```

### 5.Kỹ thuật tìm offset tiếp theo của Frame

- Yêu cầu: Ta đang ở frame đầu tiên làm sao để nhảy qua frame tiếp theo?

- Chúng ta chỉ cần xác định được kích thước (k bytes) của Frame hiện tại và di chuyển (k bytes) để tới offset của frame tiếp theo.

- Ở trên chúng ta đã biết 1 Frame sẽ gồm Header + Audio data. các thông số tính toán của Frame sẽ nằm trong Header

<img src="img/mp3frame_blocks.gif" alt="drawing" width="100"/>

Theo như [https://id3.org/mp3Frame](https://id3.org/mp3Frame) ta biết được rằng:

```
                FrameSize = 144 * BitRate / SampleRate
                   when the padding bit is cleared and
                 FrameSize = (144 * BitRate / SampleRate) + 1
                   when the padding bit is set.
```

a. ID: 1bit [12] : bit này dùng để xác định version MPEG- của file MP3

>ID=0: MPEG-2
>
>ID=1: MPEG-1

b. Layer: 2bit [13-14] : bit này dùng để xác định layer của file MP3

<table style="width: &amp;quot;150px&amp;quot"><tbody><tr>  <td><p class="line862"> 0 0 </p></td>
  <td><p class="line862"> Not defined </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-7"></span><p class="line862">0 1 </p></td>
  <td><p class="line862"> Layer III </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-8"></span><p class="line862">1 0 </p></td>
  <td><p class="line862"> Layer II </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-9"></span><p class="line862">1 1 </p></td>
  <td><p class="line862"> Layer I </p></td>
</tr>
</tbody></table>

c. BitRate: 4bit [16->19]

Bảng dưới được tính là kbit/s khi tính toán ta phải $*1000$ để trở thành bit/s

<table style="width: &amp;quot;600px&amp;quot"><tbody><tr>  <td><p class="line891"><strong> Bitrate<br>
value </strong> </p></td>
  <td><p class="line891"><strong> MPEG-1,<br>
layer I </strong> </p></td>
  <td><p class="line891"><strong> MPEG-1,<br>
layer II </strong> </p></td>
  <td><p class="line891"><strong> MPEG-1,<br>
layer III </strong> </p></td>
  <td><p class="line891"><strong> MPEG-2,<br>
layer I </strong> </p></td>
  <td><p class="line891"><strong> MPEG-2,<br>
layer II </strong> </p></td>
  <td><p class="line891"><strong> MPEG-2,<br>
layer III </strong> </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-14"></span><p class="line862"> 0 0 0 0 </p></td>
  <td colspan="6" style="&amp;quot; text-align: center; &amp;quot;"><p class="line862"> </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-15"></span><p class="line862"> 0 0 0 1 </p></td>
  <td><p class="line862"> 32 </p></td>
  <td><p class="line862"> 32 </p></td>
  <td><p class="line862"> 32 </p></td>
  <td><p class="line862"> 32 </p></td>
  <td><p class="line862"> 32 </p></td>
  <td><p class="line862"> 8 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-16"></span><p class="line862"> 0 0 1 0 </p></td>
  <td><p class="line862"> 64 </p></td>
  <td><p class="line862"> 48 </p></td>
  <td><p class="line862"> 40 </p></td>
  <td><p class="line862"> 64 </p></td>
  <td><p class="line862"> 48 </p></td>
  <td><p class="line862"> 16 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-17"></span><p class="line862"> 0 0 1 1 </p></td>
  <td><p class="line862"> 96 </p></td>
  <td><p class="line862"> 56 </p></td>
  <td><p class="line862"> 48 </p></td>
  <td><p class="line862"> 96 </p></td>
  <td><p class="line862"> 56 </p></td>
  <td><p class="line862"> 24 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-18"></span><p class="line862"> 0 1 0 0 </p></td>
  <td><p class="line862"> 128 </p></td>
  <td><p class="line862"> 64 </p></td>
  <td><p class="line862"> 56 </p></td>
  <td><p class="line862"> 128 </p></td>
  <td><p class="line862"> 64 </p></td>
  <td><p class="line862"> 32 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-19"></span><p class="line862"> 0 1 0 1 </p></td>
  <td><p class="line862"> 160 </p></td>
  <td><p class="line862"> 80 </p></td>
  <td><p class="line862"> 64 </p></td>
  <td><p class="line862"> 160 </p></td>
  <td><p class="line862"> 80 </p></td>
  <td><p class="line862"> 64 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-20"></span><p class="line862"> 0 1 1 0 </p></td>
  <td><p class="line862"> 192 </p></td>
  <td><p class="line862"> 96 </p></td>
  <td><p class="line862"> 80 </p></td>
  <td><p class="line862"> 192 </p></td>
  <td><p class="line862"> 96 </p></td>
  <td><p class="line862"> 80 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-21"></span><p class="line862"> 0 1 1 1 </p></td>
  <td><p class="line862"> 224 </p></td>
  <td><p class="line862"> 112 </p></td>
  <td><p class="line862"> 96 </p></td>
  <td><p class="line862"> 224 </p></td>
  <td><p class="line862"> 112 </p></td>
  <td><p class="line862"> 56 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-22"></span><p class="line862"> 1 0 0 0 </p></td>
  <td><p class="line862"> 256 </p></td>
  <td><p class="line862"> 128 </p></td>
  <td><p class="line862"> 112 </p></td>
  <td><p class="line862"> 256 </p></td>
  <td><p class="line862"> 128 </p></td>
  <td><p class="line862"> 64 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-23"></span><p class="line862"> 1 0 0 1 </p></td>
  <td><p class="line862"> 288 </p></td>
  <td><p class="line862"> 160 </p></td>
  <td><p class="line862"> 128 </p></td>
  <td><p class="line862"> 288 </p></td>
  <td><p class="line862"> 160 </p></td>
  <td><p class="line862"> 128 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-24"></span><p class="line862"> 1 0 1 0 </p></td>
  <td><p class="line862"> 320 </p></td>
  <td><p class="line862"> 192 </p></td>
  <td><p class="line862"> 160 </p></td>
  <td><p class="line862"> 320 </p></td>
  <td><p class="line862"> 192 </p></td>
  <td><p class="line862"> 160 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-25"></span><p class="line862"> 1 0 1 1 </p></td>
  <td><p class="line862"> 352 </p></td>
  <td><p class="line862"> 224 </p></td>
  <td><p class="line862"> 192 </p></td>
  <td><p class="line862"> 352 </p></td>
  <td><p class="line862"> 224 </p></td>
  <td><p class="line862"> 112 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-26"></span><p class="line862"> 1 1 0 0 </p></td>
  <td><p class="line862"> 384 </p></td>
  <td><p class="line862"> 256 </p></td>
  <td><p class="line862"> 224 </p></td>
  <td><p class="line862"> 384 </p></td>
  <td><p class="line862"> 256 </p></td>
  <td><p class="line862"> 128 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-27"></span><p class="line862"> 1 1 0 1 </p></td>
  <td><p class="line862"> 416 </p></td>
  <td><p class="line862"> 320 </p></td>
  <td><p class="line862"> 256 </p></td>
  <td><p class="line862"> 416 </p></td>
  <td><p class="line862"> 320 </p></td>
  <td><p class="line862"> 256 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-28"></span><p class="line862"> 1 1 1 0 </p></td>
  <td><p class="line862"> 448 </p></td>
  <td><p class="line862"> 384 </p></td>
  <td><p class="line862"> 320 </p></td>
  <td><p class="line862"> 448 </p></td>
  <td><p class="line862"> 384 </p></td>
  <td><p class="line862"> 320 </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-29"></span><p class="line862"> 1 1 1 1 </p></td>
  <td colspan="6" style="&amp;quot; text-align: center; &amp;quot;"><p class="line862"> </p></td>
</tr>
</tbody></table>

In [85]:
BitrateTBL = [
    #      MPEG1                      MPEG2
    # L1     L2      L3          L1      L2      L3
    
    
    [[0,    0,      0],         [0, 	0,	     0 ]],
    [[32,   32,     32],	    [32, 	32,	     8 ]],
    [[64,   48,     40],	    [64,	48,	     16]],
    [[96,   56,     48],	    [96, 	56,	     24]],
    [[128,	64,	    56],	    [128,	64,	     32]],
    [[160,	80,	    64],	    [160,	80,	     64]],
    [[192,	96,	    80],	    [192,	96,	     80]],
    [[224,	112,	96],	    [224,	112,	 56]],
    [[256,	128,	112],	    [256,	128,	 64]],
    [[288,	160,	128],	    [288,	160,	128]],
    [[320,	192,	160],	    [320,	192,	160]],
    [[352,	224,	192],	    [352,	224,	112]],
    [[384,	256,	224],	    [384,	256,	128]],
    [[416,	320,	256],	    [416,	320,	256]],
    [[448,	384,	320],	    [448,	384,	320]],
    [[0,    0,        0],	    [0, 	0,	     0 ]],

]

b. SampleRate: 2bit [20->21]

<table style="width: &amp;quot;255px&amp;quot"><tbody><tr>  <td><p class="line891"><strong> Frequency<br>
Value</strong> </p></td>
  <td><p class="line891"><strong>MPEG-1</strong> </p></td>
  <td><p class="line891"><strong>MPEG-2</strong> </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-34"></span><p class="line862">0 0 </p></td>
  <td><p class="line862">44100 Hz </p></td>
  <td><p class="line862">22050 Hz </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-35"></span><p class="line862">0 1 </p></td>
  <td><p class="line862">48000 Hz </p></td>
  <td><p class="line862">24000 Hz </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-36"></span><p class="line862">1 0 </p></td>
  <td><p class="line862">32000 Hz </p></td>
  <td><p class="line862">16000 Hz </p></td>
</tr>
<tr>  <td><span class="anchor" id="line-37"></span><p class="line862">1 1 </p></td>
  <td colspan="2" style="&amp;quot; text-align: center; &amp;quot;"><p class="line862"> </p></td>
</tr>
</tbody></table>

In [86]:
FrequencyTBL = [
    #   MPEG1   MPEG2
    [44100,     22050 ],
    [24000,     48000 ],
    [32000,     16000 ],
    [0,         0]           
]

c. Padding bit: 1 bit [22]

### 6.Demo

In [87]:
from bitarray import bitarray

In [88]:
# Hàm hỗ trợ:

def compute_msg_bits(msg_file, embedable_byte_length):
    with open(msg_file, 'rb') as f:
        msg = f.read()
    
    msg_bits = bitarray(); 
    msg_bits.frombytes(msg)
    if len(msg_bits) + 1 > embedable_byte_length:
        print('Cannot embed!')
        return None
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (embedable_byte_length - len(msg_bits) - 1))
    
    return list(msg_bits)

In [89]:
class MP3Steganography:
    def __init__(self,fileName) -> None:
        with open(fileName, "rb") as f:
            self.bytes_read = f.read()

        self.end=len(self.bytes_read)
        # https://id3.org/ID3v1
        # Loại bỏ ID3v1_tag
        if self.bytes_read[-128:-125].decode()=="TAG":
            self.end-=128
        # https://id3.org/id3v2.4.0-structure
        # Loại bỏ ID3v2_tag
        if self.bytes_read[0:3].decode() == "ID3":
            size = self.bytes_read[6:10]
            self.start=self.current= synchsafe2int(size) + 10 # header size = 10byte
        
        self.embedableBits = self.getEmbedableBits()

    def getEmbedableBits(self):
        result = 0
        while self.current<self.end:
            result+=1
            self.moveNextFrame()
        self.current=self.start        
        return result * 5 # five bits can embed

    def moveNextFrame(self):
        # https://id3.org/mp3Frame
        currentHeader = self.bytes_read[self.current:self.current+5]    # take 4 bytes
        MPEGVersion = currentHeader[1] >> 3 & 1                         # bit index 12 (8*2-1 - 3 & 1)
        #=1 mpeg 1, =0 mpeg2

        Layer = currentHeader[1] >> 1 & 3                               # bit index 13-14 (8*2-1 -1 & 11)
        #01: 1 layer3
        #10: 2 layer2
        #11: 3 layer1

        BitRate = currentHeader[2] >> 4 & 15                            # bit index 16-19 (8*3-1 -4 & 1111)
        Frequency = currentHeader[2] >> 2 & 3                           # bit index 20-21 (8*3-1 -2 & 11)
        Pad = currentHeader[2] >> 1 & 1                                 # bit index 22 (8*3-1 -1 & 1)

        _Frequency = FrequencyTBL[Frequency][1-MPEGVersion] # Swap to MPEG-1 and MPEG-2
        _BitRate = BitrateTBL[BitRate][1-MPEGVersion][4-Layer-1] # Swap to MPEG-1 and MPEG-2 and Layer 1,2,3
        
        FrameSize = 144 * _BitRate * 1000 / _Frequency
        if Pad:
            FrameSize += 1

        self.current += int(FrameSize)

    def embed(self, bits, stego_file):
        b=0
        stegobyte = list(self.bytes_read)
        while self.current<self.end:
            # bit24
            tmp1 = stegobyte[self.current + 2]
            stegobyte[self.current + 2] = tmp1 & 0xFE | bits[b] # giữ lại 7 bit đầu trừ 1 bit cuối = bit cần chèn
            b+=1 
            # bit 29,30,31,32
            tmp2 = stegobyte[self.current + 3]
            bitcombibe = 0
            for _ in range(4):
                bitcombibe = (bitcombibe << 1) | bits[b]
                b+=1
            stegobyte[self.current + 3] = tmp2 & 0xF0 | bitcombibe # giữ lại 4 bit đầu trừ 4 bit cuối = 4 bit cần chèn
            self.moveNextFrame()

        with open(stego_file, 'wb') as f:
            f.write(bytearray(stegobyte)) 
        
    def extract(self, extract_file):
        extr_msg_bits = bitarray()
        while self.current<self.end:
            tmp1 = self.bytes_read[self.current + 2]
            extr_msg_bits.append(tmp1 & 1)
            tmp2 = self.bytes_read[self.current + 3]
            for i in range(3,-1,-1):
                extr_msg_bits.append(tmp2 >> i & 1)
            self.moveNextFrame()

        extr_msg_bits = extr_msg_bits[:extr_msg_bits.to01().rfind('1')]
        
        with open(extract_file, 'wb') as f:
            f.write(extr_msg_bits.tobytes())


**TEST**

EMBED

In [90]:
cover_file = "cover.mp3"
secret_file = "big.jfif"
tool = MP3Steganography(cover_file)
msg_bit = compute_msg_bits(secret_file,tool.embedableBits)
assert msg_bit==None

Cannot embed!


In [91]:
cover_file = "cover.mp3"
correct_stego_file = "correct_stego.mp3"
stego_file = "stego.mp3"
secret_file = "small.jfif"
tool = MP3Steganography(cover_file)
msg_bit = compute_msg_bits(secret_file,tool.embedableBits)
if msg_bit:
    tool.embed(msg_bit,stego_file)

with open(correct_stego_file, 'rb') as f:
    correct_stego = f.read()
with open(stego_file, 'rb') as f:
    stego = f.read()

assert correct_stego == stego

EXTRACT

In [92]:
stego_file = "stego.mp3"
correct_extract_file = "small.jfif"
extract_file = "small_extract.jfif"

tool = MP3Steganography(stego_file)
tool.extract(extract_file)

with open(correct_extract_file, 'rb') as f:
    correct_extract = f.read()
with open(extract_file, 'rb') as f:
    extract = f.read()

assert correct_extract == extract